In [57]:
import numpy as np

def analyze_portfolio(**kwargs):
    
    '''
    kwargs made up of
    ------
    number_stocks (int)
    risk_free_asset_return (int)
    b_1 (int) = Model Parameter #1
    b_2 (int) = Model Parameter #2
    F_1 {mean, stdev} = Factor 1
    F_2 {mean, stdev} = Factor 2
    e_1 {mean, stdev}
    '''
    k = kwargs
    
    def get_individual_stock_return(x):
        return  k["b_1"](x)*k["F_1"]["mean"]+\
                k["b_2"](x)*k["F_2"]["mean"]+\
                k["e_1"]["mean"]
    
    def get_individual_stock_stdev(x):
        return  (k["b_1"](x)**2)*(k["F_1"]["stdev"]**2)+\
                (k["b_2"](x)**2)*(k["F_2"]["stdev"]**2)+\
                (k["e_1"]["stdev"]**2)
    
    stock_number = np.arange(1,k["number_stocks"]+1,1)
    
    stock_weights = np.full((1, k["number_stocks"]), 1/k["number_stocks"])
    
    get_return = np.vectorize(get_individual_stock_return)
    stock_returns = get_return(stock_number)
    
    portfolio_return = stock_weights @ stock_returns.transpose()
    risk_premium = portfolio_return - kwargs["risk_free_asset_return"]
    
    get_variance = np.vectorize(get_individual_stock_stdev)
    stock_variances = get_variance(stock_number)
    
    def construct_covariance_matrix():
        matrix = np.zeros( (k["number_stocks"], k["number_stocks"]) )
        for i in range(1,k["number_stocks"]+1):
            for j in range(1,k["number_stocks"]+1):
                if i == j:
                    matrix[i-1][j-1] = stock_variances[i-1]
                else:
                    matrix[i-1][j-1] = \
                    (k["b_1"](i)*k["b_1"](j))*(k["F_1"]["stdev"]**2)+\
                    (k["b_2"](i)*k["b_2"](j))*(k["F_2"]["stdev"]**2)
        return matrix
    
    covariance_matrix = construct_covariance_matrix()
    portfolio_var = stock_weights @ (stock_weights @ covariance_matrix).transpose()
    portfolio_stdev = portfolio_var**(1/2)
    portfolio_sr = risk_premium/portfolio_stdev
    print(f"The Portfolio Return is {portfolio_return}")
    print(f"The Portfolio Variance is {portfolio_var}")
    print(f"The Portfolio Stdev is {portfolio_stdev}")
    print(f"The Portfolio SR (Sharpe Ratio) is {portfolio_sr}")
          
    def get_covariance(i):
          return (covariance_matrix[i]@stock_weights.transpose())[0]
    
    stock_to_portfolio_covariances = [get_covariance(i) for i in range(k["number_stocks"])]
    print(  f"The Covariances For The First Five Individual Stocks " + \
            f"to the Portfolio are" +\
            f"{np.around(stock_to_portfolio_covariances[0:5],2)}")

    def get_stock_rrr(i):
        r_i = stock_returns[i-1]
        excess_return_i = r_i - kwargs["risk_free_asset_return"]
        cov_i = stock_to_portfolio_covariances[i-1]
        return excess_return_i/(cov_i/portfolio_stdev)
          
    get_rrr = np.vectorize(get_stock_rrr)
    stock_rrrs = get_rrr(stock_number)
    
    print(  f"The RRRs For The First Five Individual Stocks" +\
            f"to the Portfolio are {np.around(stock_rrrs[0:5],2)}")
    
    inverted_covariances = np.linalg.inv(covariance_matrix)
    stock_excess_returns = stock_returns - k["risk_free_asset_return"]
    ones = np.ones((1,k["number_stocks"]))
    lambda_scalar = 1/(stock_excess_returns@inverted_covariances@ones.transpose())
    print(lambda_scalar)
    tangency_weights = lambda_scalar*(inverted_covariances@stock_excess_returns.transpose())
    
    print(f"The First Five Tangency Weights are" +\
          f"{np.around(tangency_weights[0:5],2)}")
    
    optimal_return = tangency_weights @ stock_returns.transpose()
    optimal_risk_premium = optimal_return - kwargs["risk_free_asset_return"]
    optimal_var = tangency_weights @ (tangency_weights @ covariance_matrix).transpose()
    optimal_stdev = optimal_var**(1/2)
    optimal_sr = optimal_risk_premium/optimal_stdev
    print(f"Optimal Sharpe Ratio is {optimal_sr}")

### Problem 1

Suppose there are 10 stocks in the market. Each stock's expected returns follows a two-factor model: r_i = b_1*F_1 + b_2,i*F_2 + e

Model parameters:
b_1 = 10
b_2,i = i
F_1 has zero mean and 30% standard deviation
Assume F_1, F_2, and e are independent from each other
There is a risk-free asset with guaranteed return of 0.75%.

In [58]:
analyze_portfolio(
        number_stocks = 10,
        risk_free_asset_return = 0.0075,
        b_1 = lambda x: 10,
        b_2 = lambda x: x,
        F_1 = {"mean":0, "stdev":0.01},
        F_2 = {"mean":0.01, "stdev":0.01},
        e_1 = {"mean":0, "stdev":0.3}
) # SR 0.32006313

The Portfolio Return is [0.055]
The Portfolio Variance is [[0.022025]]
The Portfolio Stdev is [[0.14840822]]
The Portfolio SR (Sharpe Ratio) is [[0.32006313]]
The Covariances For The First Five Individual Stocks to the Portfolio are[0.02 0.02 0.02 0.02 0.02]
The RRRs For The First Five Individual Stocksto the Portfolio are [0.02 0.09 0.16 0.23 0.29]
[0.50769915]
The First Five Tangency Weights are[-0.11 -0.06 -0.02  0.03  0.08]
Optimal Sharpe Ratio is 0.41040497137944043


### Problem 2

Suppose there are risky 300 stocks in the market, and their returns follow a single-factor model. Return on stock is given by
r_i = r_expected + b*F + e

where r_expected (expected return) is 8%
where b (factor loading) is 0.298
where F has zero mean and unit variance
where standard deviation of the idiosyncratic shock is 25%
Assume that the risk-free rate (r_f) is 3%

Consider Portfolio P with equal allocation into each of the stocks and zero weight on the risk-free asset.

In [59]:
analyze_portfolio(
        number_stocks = 300,
        risk_free_asset_return = 0.03,
        b_1 = lambda x: 1,
        b_2 = lambda x: 0.298,
        F_1 = {"mean":0.08, "stdev":0},
        F_2 = {"mean":0, "stdev":1},
        e_1 = {"mean":0, "stdev":0.25}
)
# CONFIRMED ANSWERS
# portfolio -> r: 0.08, variance: 0.089, SR 0.16758877
# optimal sharpe ratio -> 0.4104

The Portfolio Return is [0.08]
The Portfolio Variance is [[0.08901233]]
The Portfolio Stdev is [[0.29834935]]
The Portfolio SR (Sharpe Ratio) is [[0.16758877]]
The Covariances For The First Five Individual Stocks to the Portfolio are[0.09 0.09 0.09 0.09 0.09]
The RRRs For The First Five Individual Stocksto the Portfolio are [0.17 0.17 0.17 0.17 0.17]
[1.78024667]
The First Five Tangency Weights are[0. 0. 0. 0. 0.]
Optimal Sharpe Ratio is 0.16758876923487015
